# get_video_info

In [ ]:
import subprocess
import datetime
import os

def get_video_info(input_path):
    # ffprobe command to get video codec, dimensions, bitrate, fps, duration, and file size
    cmd = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=codec_name,width,height,bit_rate,r_frame_rate',
        '-show_entries', 'format=duration,size',  # Including format=size to get the file size
        '-of', 'default=noprint_wrappers=1:nokey=1',
        input_path
    ]

    # Execute the ffprobe command
    output = subprocess.check_output(cmd).decode('utf-8').strip().split('\n')

    # Extracting the information from the output
    video_codec = output[0]
    width = output[1]
    height = output[2]
    bitrate = output[4]
    fps_frac = output[3]
    duration = float(output[5])
    file_size = output[6]  # Corrected file size index

    # Calculating FPS from the fractional representation
    fps = "N/A"
    if fps_frac:
        try:
            numerator, denominator = map(int, fps_frac.split('/'))
            fps = round(float(numerator) / float(denominator), 2)
        except ValueError:
            # Handle the case where the FPS is not a fraction (which is rare)
            fps = float(fps_frac)

    # Formatting dimensions
    dimensions = f"{width}x{height}"

    # Formatting bitrate as Mb/s if bitrate is available
    bitrate_mbps = "N/A"
    if bitrate and bitrate.isdigit():
        bitrate_mbps = round(int(bitrate) / 1e6, 1)

    # Formatting file size as MB
    size_mb = round(int(file_size) / 1e6, 1)

    # Formatting duration as HH:MM:SS
    duration_str = str(datetime.timedelta(seconds=duration))

    # Returning the information as a tuple
    return video_codec, dimensions, bitrate_mbps, fps, duration_str, size_mb

testfile="./test/DJI_0021.MP4"
testdir="./test"

get_video_info(testfile)


# get_export_settings

In [ ]:
def get_export_settings(input_path):
    # Use the get_video_info function to get video parameters
    video_codec, dimensions, bitrate_mbps, fps, duration_str, size_mb = get_video_info(input_path)
    
    # Determine resolution based on dimensions
    width, height = map(int, dimensions.split('x'))
    resolution = None
    if (width >= 3840 and height >= 2160) or (width >= 2160 and height >= 3840):  # Handling both landscape and portrait 4K
        resolution = '4k'
    elif (width >= 1920 and height >= 1080) or (width >= 1080 and height >= 1920):  # Handling both landscape and portrait 1080p
        resolution = '1080p'

    # Round fps to the nearest whole number to match against '30' or '60'
    frame_rate = str(round(fps / 30) * 30)  # This will convert fps to either '30' or '60'

    # Use resolution and frame rate to get the correct settings from the dictionary
    settings = ffmpeg_settings.get((resolution, frame_rate))

    return settings

# get_export_settings(testfile)






In [ ]:
def bitrate_to_size(duration_str, bitrate_mbps):
    """
    Estimates the file size in MB based on the video duration and bitrate.
    The duration_str is expected to be in HH:MM:SS format.
    The bitrate_mbps is expected to be in Mbps.
    """
    # Convert HH:MM:SS to total seconds
    hours, minutes, seconds = map(float, duration_str.split(':'))
    total_seconds = hours * 3600 + minutes * 60 + seconds
    
    # Calculate the estimated file size in MB
    if bitrate_mbps != "N/A":
        file_size_mb = (bitrate_mbps * total_seconds * 0.125)
    else:
        file_size_mb = "N/A"
    
    return round(file_size_mb, 2) if file_size_mb != "N/A" else file_size_mb

bitrate_to_size("00:01:00", 5.0)


# estimate_new_file_size

In [ ]:
def estimate_new_file_size(input_path):
    global settings
    # Get video information
    video_codec, dimensions, bitrate_mbps, fps, duration_str, size_mb = get_video_info(input_path)

    # Get export settings
    settings = get_export_settings(input_path)

    # If no matching settings found, return "N/A"
    if not settings:
        return "N/A"

    # Get bitrate from settings
    bitrate_mbps = float(settings.get('bitrate'))

    # Estimate new file size
    new_file_size_mb = round(bitrate_to_size(duration_str, bitrate_mbps),1)

    # Calculate compression ratio
    compression_ratio = 1 - (new_file_size_mb / size_mb)

    return new_file_size_mb, bitrate_mbps, size_mb, f'{round(compression_ratio * 100, 1)}%'

estimate_new_file_size(testfile)

# parse_videos

In [ ]:
def parse_videos(input_path):
    extensions = ['.mp4', '.mkv', '.avi', 'mov']
    videos = [os.path.join(input_path, f) for f in os.listdir(input_path) if any(f.lower().endswith(ext) for ext in extensions)]
    return videos

parse_videos(testdir)

# save_new_filename

In [ ]:
def save_new_filename(input_filedir):
    dir_name = os.path.dirname(input_filedir)
    file_name = os.path.splitext(os.path.basename(input_filedir))[0]
    extension = '.mp4'  # Since we want to convert all files to mp4
    counter = 1

    output_filedir = os.path.join(dir_name, f"{file_name}{extension}")

    while os.path.exists(output_filedir):
        output_filedir = os.path.join(dir_name, f"{file_name} {counter}{extension}")
        counter += 1
    return output_filedir

# save_new_filename(testfile)

# get_rating

In [ ]:
def get_rating(file_path):
    # Command to get the rating using exiftool
    rating_cmd = ['exiftool', '-XMP:Rating', file_path]

    # Run the command and capture the output
    completed_process = subprocess.run(rating_cmd, stdout=subprocess.PIPE, text=True)

    # Check if the command was successful
    if completed_process.returncode == 0:
        # Extract the rating from the output
        output = completed_process.stdout.strip()

        # Assume output format: "XMP:Rating                        : <RatingValue>"
        # Split the output and get the last element, which should be the rating
        rating = output.split(':')[-1].strip()

        return rating
    else:
        # Handle error if exiftool failed
        print("Error: exiftool did not complete successfully.")
        return None

# GUI

In [ ]:
# import tkinter as tk
# from tkinter import filedialog
# import os

# # Define global variables
# video_dir = ""
# video_list = []
# cb_vars = []

# # Create the main window
# root = tk.Tk()
# root.title("Video Converter")

# # Create a function to select a folder
# def select_folder():
#     global video_dir
#     video_dir = filedialog.askdirectory()
#     scan_folder()

# # Create a function to scan the folder for video files
# def scan_folder():
#     global video_list
#     video_list = parse_videos(video_dir)
#     display_videos()

# # Create a function to display the list of videos with checkboxes
# def display_videos():
#     global video_list, cb_vars
#     for video in video_list:
#         var = tk.BooleanVar()
#         var.set(True)
#         cb_vars.append(var)
#         cb = tk.Checkbutton(root, text=video, variable=var)
#         cb.pack()

# # Create a function to process the selected videos
# def process_videos():
#     global video_list, cb_vars
#     for i, video in enumerate(video_list):
#         if cb_vars[i].get():
#             convert_video_handbrake(os.path.join(video_dir, video))

# # Create the GUI
# select_button = tk.Button(root, text="Select Folder", command=select_folder)
# select_button.pack()

# process_button = tk.Button(root, text="Process Videos", command=process_videos)
# process_button.pack()

# root.mainloop()


# Print all functions

In [ ]:
# This will print out a list of all user-defined functions in the notebook
functions_list = [f for f in globals().values() if callable(f) and f.__module__ == '__main__']
for func in functions_list:
    print(func.__name__)

In [ ]:
# import tkinter as tk
# from tkinter import ttk
# import os

# def populate_tree(tree, path):
#     for entry in parse_videos(path):
#         rating = get_rating(entry)
#         size = os.stat(entry).st_size
#         if size >= 1024**3:
#             size_str = f"{size / 1024**3:.1f} GB"
#         elif size >= 1024**2:
#             size_str = f"{size / 1024**2:.1f} MB"
#         else:
#             size_str = f"{size / 1024:.1f} KB"
#         tree.insert('', 'end', values=(entry, size_str, rating))

# root = tk.Tk()
# tree = ttk.Treeview(root, columns=('Name', 'Size', 'Rating'), show='headings')
# tree.heading('Name', text='File Name')
# tree.heading('Size', text='File Size (bytes)')
# tree.heading('Rating', text='Rating')
# tree.pack(fill=tk.BOTH, expand=True)

# populate_tree(tree, testdir)

# root.mainloop()


In [ ]:
# from PyQt5.QtWidgets import QApplication, QTreeWidget, QTreeWidgetItem, QCheckBox, QVBoxLayout, QWidget
# from PyQt5.QtCore import Qt

# class CheckableTreeWidget(QTreeWidget):
#     def __init__(self):
#         super().__init__()
#         self.setColumnCount(3)
#         self.setHeaderLabels(['ID', 'Name', 'Value'])

#         # Populate the tree with items
#         self.populate_tree()

#     def populate_tree(self):
#         # Dummy data for the treeview
#         data = [("Item 1", "Value 1"), ("Item 2", "Value 2"), ("Item 3", "Value 3")]

#         for i, (name, value) in enumerate(data, 1):
#             item = QTreeWidgetItem([str(i), name, value])
#             self.addTopLevelItem(item)
#             # Create a checkbox and set its state
#             checkbox = QCheckBox()
#             checkbox.stateChanged.connect(lambda state, id=i: self.on_checked(id, state))
#             # Place the checkbox in the second column
#             self.setItemWidget(item, 1, checkbox)

#     def on_checked(self, id, state):
#         print(f"Checkbox for row {id} is {'checked' if state == Qt.Checked else 'unchecked'}")


# class MainWindow(QWidget):
#     def __init__(self):
#         super().__init__()

#         self.tree = CheckableTreeWidget()

#         # Layout
#         layout = QVBoxLayout()
#         layout.addWidget(self.tree)
#         self.setLayout(layout)


# if __name__ == '__main__':
#     app = QApplication([])
#     window = MainWindow()
#     window.show()
#     app.exec_()


In [ ]:
# import sys
# from PyQt5.QtWidgets import QApplication, QMainWindow, QTableWidget, QTableWidgetItem, QCheckBox, QVBoxLayout, QWidget
# from PyQt5.QtCore import Qt


# class MainWindow(QMainWindow):
#     def __init__(self):
#         super().__init__()

#         self.initializeUI()

#     def initializeUI(self):
#         # Set the main window properties
#         self.setWindowTitle('PyQt5 Table with Checkboxes')
#         self.setGeometry(100, 100, 400, 200)

#         # Create the table
#         self.tableWidget = QTableWidget()
#         self.tableWidget.setRowCount(3)
#         self.tableWidget.setColumnCount(3)

#         # Set the table headers
#         self.tableWidget.setHorizontalHeaderLabels(['ID', 'Name', 'Active'])

#         # Add items to the table and checkboxes
#         for i in range(3):
#             # ID
#             self.tableWidget.setItem(i, 0, QTableWidgetItem(str(i + 1)))
#             # Name
#             self.tableWidget.setItem(i, 1, QTableWidgetItem(f'Item {i + 1}'))
#             # Checkboxes
#             checkBoxWidget = QWidget()
#             checkBox = QCheckBox()
#             layoutCheckBox = QVBoxLayout(checkBoxWidget)
#             layoutCheckBox.addWidget(checkBox)
#             layoutCheckBox.setAlignment(Qt.AlignCenter)
#             layoutCheckBox.setContentsMargins(0,0,0,0)
#             checkBoxWidget.setLayout(layoutCheckBox)
#             self.tableWidget.setCellWidget(i, 2, checkBoxWidget)

#         self.setCentralWidget(self.tableWidget)

# if __name__ == '__main__':
#     app = QApplication(sys.argv)
#     mainWin = MainWindow()
#     mainWin.show()
#     sys.exit(app.exec_())


In [1]:
ffmpeg_settings = {
    '4k': {
        '30': {
            'vt_h265': {
                'LQ': '25',
                'HQ': '60'
            },
        },
        '60': {
            'vt_h265': {
                'LQ': '30',
                'HQ': '70'
            },
        },
        '120': {
            'vt_h265': {
                'LQ': '50',  # These are hypothetical values
                'HQ': '100'  # Adjust them as per the actual quality settings required
            },
        },
    },
    '1080p': {
        '30': {
            'vt_h265': {
                'LQ': '8',
                'HQ': '15'
            },
        },
        '60': {
            'vt_h265': {
                'LQ': '10',
                'HQ': '20'
            },
        },
        '120': {
            'vt_h265': {
                'LQ': '20',  # These are hypothetical values
                'HQ': '40'   # Adjust them as per the actual quality settings required
            },
        },
    }
}

In [14]:
resolution = "4k"
frame_rate = "30"
codec = "vt_h265"
quality = "LQ"


settings = {"bitrate": ffmpeg_settings.get(resolution, {}).get(str(frame_rate), {}).get(codec, {}).get(quality), "codec":list(ffmpeg_settings.get(resolution, {}).get(str(frame_rate)).keys())[0]}
settings


{'bitrate': '25', 'codec': 'vt_h265'}